In [1]:
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
#   except RuntimeError as e:
#     print(e)

### utils

In [2]:
def class_mae(y_true, y_pred):
    return K.mean(
        K.abs(
            K.argmax(y_pred, axis=-1) - K.argmax(y_true, axis=-1)
        ),
        axis=-1
    )


In [3]:
# print(keras.__version__)
print(tf.__version__)

2.7.0


In [4]:
# print("Keras version :",keras.__version__)

# Convert model to new tensorflow

In [6]:
with open('json_config/CRNN_config.json') as json_file:
    json_config = json_file.read()

model = tf.compat.v1.keras.models.model_from_json(
    json_config,
    custom_objects={
        'class_mae': class_mae,
        'exp': K.exp,
    }
)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero1 (ZeroPadding2D)       (None, 1, 500, 201)       0         
                                                                 
 conv1 (Conv2D)              (None, 64, 498, 199)      640       
                                                                 
 conv2 (Conv2D)              (None, 32, 496, 197)      18464     
                                                                 
 pool1 (MaxPooling2D)        (None, 32, 165, 65)       0         
                                                                 
 conv3 (Conv2D)              (None, 128, 163, 63)      36992     
                                                                 
 conv4 (Conv2D)              (None, 64, 161, 61)       73792     
                                                                 
 pool2 (MaxPooling2D)        (None, 64, 53, 20)       

In [7]:
model.load_weights("../CountNet/models/CRNN.h5")

# test

In [150]:
from time import time
RUN_FOR = 100

In [31]:
model = tf.keras.models.load_model("models/CRNN")

In [33]:
# tf random tensor
tf_random_tensor = tf.random.uniform((1, 1, 500, 201))
print("Input shape", tf_random_tensor.shape)
print("Output shape", model.predict(tf_random_tensor).shape)
tik = time()
for _ in range(RUN_FOR):
    model.predict(tf_random_tensor)
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Input shape (1, 1, 500, 201)
Output shape (1, 11)
Duration :5.19s


In [47]:
model = tf.keras.models.load_model("models/F-CRNN")

In [48]:
# tf random tensor
tf_random_tensor = tf.random.uniform((1, 500, 201))
print("Input shape", tf_random_tensor.shape)
print("Output shape", model.predict(tf_random_tensor).shape)
tik = time()
for _ in range(RUN_FOR):
    model.predict(tf_random_tensor)
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Input shape (1, 500, 201)
Output shape (1, 11)
Duration :5.02s


# ONNX

In [163]:
import onnxruntime as ort
import numpy as np

tf_random_tensor = tf.random.uniform((1,1, 500, 201))


ort_sess = ort.InferenceSession('CRNN_ONNX.onnx')
outputs = ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
# Print Result
result = outputs[0].argmax(axis=1)
print("Activations :", outputs[0][0])
print("label :",result[0])

tik = time()
for _ in range(RUN_FOR):
    ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Activations : [9.8316780e-12 2.9691032e-01 1.8009843e-03 1.6248560e-13 1.2032196e-19
 1.4747555e-19 3.9625079e-14 3.3056147e-09 3.8585404e-06 5.7177003e-03
 6.9556713e-01]
label : 10
Duration :2.18s


In [164]:
import onnxruntime as ort
import numpy as np

tf_random_tensor = tf.random.uniform((1, 500, 201))


ort_sess = ort.InferenceSession('F-CRNN_ONNX.onnx')
outputs = ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
# Print Result
result = outputs[0].argmax(axis=1)
print("Activations :", outputs[0][0])
print("label :",result[0])

tik = time()
for _ in range(RUN_FOR):
    ort_sess.run([], {'zero1_input': tf_random_tensor.numpy()})
tok = time()
print("Duration :{:.2f}s".format(tok-tik)) 

Activations : [9.24522769e-09 3.23386266e-05 2.85154317e-10 3.53782013e-14
 1.06292259e-15 3.63865296e-14 1.09060656e-10 2.83103816e-07
 6.89089138e-05 1.53245600e-02 9.84573901e-01]
label : 10
Duration :0.04s
